# GitHub Issues Query

This notebook lists issues (tickets) from a GitHub repository.

Setup:
1. Duplicate `.env.example` to `.env` in this folder
2. Fill `GITHUB_TOKEN`, `GITHUB_OWNER`, `GITHUB_REPO` (and `GITHUB_API_URL` if Enterprise)
3. Run the cells



## Fetch all issues from the repository

In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv
import requests
import pandas as pd

# Load environment variables from .env if present
load_dotenv(dotenv_path=Path(".env"), override=False)

GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
GITHUB_OWNER = os.getenv("GITHUB_OWNER")
GITHUB_REPO = os.getenv("GITHUB_REPO")
GITHUB_API_URL = os.getenv("GITHUB_API_URL", "https://api.github.com")

missing = [name for name, val in {
    "GITHUB_TOKEN": GITHUB_TOKEN,
    "GITHUB_OWNER": GITHUB_OWNER,
    "GITHUB_REPO": GITHUB_REPO,
}.items() if not val]

if missing:
    raise RuntimeError(f"Missing required env vars: {', '.join(missing)}. Copy .env.example to .env and fill them.")

session = requests.Session()
session.headers.update({
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28",
})

repo_issues_url = f"{GITHUB_API_URL}/repos/{GITHUB_OWNER}/{GITHUB_REPO}/issues"


def fetch_issues(state: str = "open", labels: str | None = None, per_page: int = 50, max_pages: int = 5):
    params = {"state": state, "per_page": per_page}
    if labels:
        params["labels"] = labels

    issues = []
    url = repo_issues_url
    for _ in range(max_pages):
        resp = session.get(url, params=params)
        resp.raise_for_status()
        page_items = resp.json()
        # GitHub returns PRs in this endpoint as well; filter to issues only
        page_issues = [it for it in page_items if "pull_request" not in it]
        issues.extend(page_issues)
        # Pagination via Link header
        next_url = None
        if "link" in resp.headers:
            for part in resp.headers["link"].split(","):
                seg, rel = part.split(";")
                if 'rel="next"' in rel:
                    next_url = seg.strip()[1:-1]
                    break
        if not next_url:
            break
        url, params = next_url, None  # next already encodes params
    return issues

issues = fetch_issues(state="open", labels=None, per_page=50, max_pages=5)
print(f"Fetched {len(issues)} issues from {GITHUB_OWNER}/{GITHUB_REPO}")

df = pd.DataFrame([
    {
        "number": it.get("number"),
        "title": it.get("title"),
        "state": it.get("state"),
        "created_at": it.get("created_at"),
        "updated_at": it.get("updated_at"),
        "user": (it.get("user") or {}).get("login"),
        "assignees": ",".join([a.get("login") for a in (it.get("assignees") or [])]),
        "labels": ",".join([l.get("name") for l in (it.get("labels") or [])]),
        "url": it.get("html_url"),
    }
    for it in issues
])

df.head(10)


Fetched 49 issues from TheSoftwareDevGuild/TheGuildGenesis


,number,title,state,created_at,updated_at,user,assignees,labels,url
0,154,Description field of badges should be bytes no...,open,2025-12-14T10:45:06Z,2025-12-15T16:57:10Z,joelamouche,V-Vaal,"good first issue,solidity,40pts,foundry,web3",https://github.com/TheSoftwareDevGuild/TheGuil...
1,153,Allow batch attestation creation in the front end,open,2025-12-14T10:39:31Z,2025-12-14T10:39:31Z,joelamouche,,"good first issue,front end,react,ux,typescript",https://github.com/TheSoftwareDevGuild/TheGuil...
2,150,Implement JWT - Front end flow,open,2025-12-08T10:36:57Z,2025-12-08T12:40:22Z,joelamouche,tusharshah21,"front end,react,typescript",https://github.com/TheSoftwareDevGuild/TheGuil...
3,146,Figure out how to admin delete profile,open,2025-12-03T17:26:37Z,2025-12-07T06:52:27Z,joelamouche,,"good first issue,back-end,planning",https://github.com/TheSoftwareDevGuild/TheGuil...
4,145,I think we should allow duplicate attestations,open,2025-12-03T15:18:42Z,2025-12-03T15:18:42Z,joelamouche,,"solidity,planning",https://github.com/TheSoftwareDevGuild/TheGuil...
5,144,Implement Upgradable pattern for TheGuildAttes...,open,2025-12-03T10:28:34Z,2025-12-03T10:28:55Z,joelamouche,,"good first issue,solidity,foundry",https://github.com/TheSoftwareDevGuild/TheGuil...
6,143,TheGuildAttestationResolver should have admin ...,open,2025-12-03T10:27:07Z,2025-12-03T10:27:51Z,joelamouche,,"good first issue,solidity,foundry",https://github.com/TheSoftwareDevGuild/TheGuil...
7,142,Add contributor leaderboard,open,2025-11-24T15:04:07Z,2025-12-13T17:03:24Z,joelamouche,adityagupta0251,"good first issue,front end,react,typescript,80pts",https://github.com/TheSoftwareDevGuild/TheGuil...
8,141,"Add a new leaderboard page, with link in the s...",open,2025-11-24T15:01:00Z,2025-12-14T10:34:19Z,joelamouche,adityagupta0251,"good first issue,front end,react,typescript,20...",https://github.com/TheSoftwareDevGuild/TheGuil...
9,140,Create top badge owner leaderboard,open,2025-11-24T14:59:35Z,2025-11-24T15:02:16Z,joelamouche,,"good first issue,front end,react,typescript,40pts",https://github.com/TheSoftwareDevGuild/TheGuil...


In [20]:
# Fetch all closed tickets
closed_tickets = fetch_issues(state="closed")

## Analyze closed tickets

In [21]:
# analyze closed tickets
# we want to get the following stats about all closed tickets: 
# - how many different contributors, 
# - how many different closed tickets, 
# - how many tags associated with the different tickets (total sum)
# - total sum of contributions points associated with tickets (see tags)
import re
import json


def get_contributions_from_tags(tags):
    # filter out the tags that don't look like xpts (where x is a number)
    tags = [tag for tag in tags if re.match(r'^\d+pts$', tag.get("name"))]
    # We want to sum the numbers
    return sum(int(tag.get("name").split("pts")[0]) for tag in tags)

def analyze_closed_tickets(closed_tickets):
    
    # Initialize dictionaries to store unique contributors and tags
    contributors = set()
    tags = set()
    total_contributions = 0
    
    # Process each closed ticket
    for ticket in closed_tickets:
        # Get the user who closed the ticket
        user = ticket.get("user", {}).get("login")
        if user:
            contributors.add(user)
        
        # Get the tags associated with the ticket
        for label in ticket.get("labels", []):
            tags.add(label.get("name"))
            
        # Get the contributions points from the tags
        contributions = get_contributions_from_tags(ticket.get("labels", []))
        total_contributions += contributions
    
    return {
        "contributors": len(contributors),
        "tags": len(tags),
        "total_contribution_tokens": total_contributions,
        "number_of_closed_tickets": len(closed_tickets),
    }
# analyze
stats = analyze_closed_tickets(closed_tickets)

# pretty print stats
print(json.dumps(stats, indent=4))
    

{
    "contributors": 4,
    "tags": 28,
    "total_contribution_tokens": 1040,
    "number_of_closed_tickets": 42
}


## Generate Attestations from Issues


### fetch all issues

In [29]:
import json
import re
from datetime import datetime
from collections import defaultdict

# Fetch all issues (only closed) to get complete picture
all_issues = fetch_issues(state="closed", per_page=100, max_pages=10)
print(f"Fetched {len(all_issues)} total issues (closed)")

Fetched 42 total issues (closed)


### Extract Labels

In [23]:
# Extract unique labels/badges from all issues
all_labels = set()
for issue in all_issues:
    for label in issue.get("labels", []):
        label_name = label.get("name", "")
        # Filter out point labels (e.g., "10pts", "5pts") as they're not badges
        if label_name and not re.match(r'^\d+pts$', label_name):
            all_labels.add(label_name)

badges_to_create = sorted(list(all_labels))
print(f"\n{'='*60}")
print(f"BADGES/LABELS THAT NEED TO BE CREATED ({len(badges_to_create)}):")
print(f"{'='*60}")
for badge in badges_to_create:
    print(f"  - {badge}")
print(f"{'='*60}\n")



BADGES/LABELS THAT NEED TO BE CREATED (23):
  - back-end
  - blockchain
  - bug
  - db
  - design
  - discord-bot
  - documentation
  - foundry
  - front end
  - good first issue
  - hacktoberfest
  - in progress
  - jupyter-notebook
  - nodejs
  - onlydust-wave
  - planning
  - python
  - react
  - rust
  - solidity
  - typescript
  - ux
  - wagmi



I'd rather dirrectly process this list into ai chat manually

#### Non skill labels to skip

In [24]:
non_skill_badges = [
  "good first issue",
  "hacktoberfest",
  "in progress",
  "bug",
  "onlydust-wave",
  "planning",
]

#### git_label_to_badge

In [25]:
git_label_to_badge = {
  "rust": "Rust",
  "solidity": "Solidity",
  "typescript": "TypeScript",
  "python": "Python",
  "apis": "APIs",

  "back-end": "Back-end",
  "backend": "Back-end",

  "blockchain": "Blockchain",

  "db": "DB",
  "database": "DB",
  "databases": "DB",

  "design": "Design",

  "discord-bot": "Discord-bot",
  "discord bot": "Discord-bot",

  "documentation": "Documentation",
  "docs": "Documentation",

  "foundry": "Foundry",

  "front end": "Front end",
  "frontend": "Front end",
  "front-end": "Front end",

  "jupyter-notebook": "Jupyter-notebook",
  "jupyter notebook": "Jupyter-notebook",

  "nodejs": "Nodejs",
  "node.js": "Nodejs",
  "node": "Nodejs",

  "onlydust-wave": "Onlydust-wave",

  "planning": "Planning",

  "react": "React",

  "ux": "UX",
  "user-experience": "UX",

  "wagmi": "Wagmi"
}

### Get github handles from API

In [26]:
# Optionally fetch profiles from backend API to map GitHub usernames to addresses
BACKEND_API_URL = os.getenv("BACKEND_API_URL", "http://localhost:3000")
print(BACKEND_API_URL)
github_to_address = {}

try:
    profiles_resp = requests.get(f"{BACKEND_API_URL}/profiles", timeout=5)
    if profiles_resp.status_code == 200:
        profiles = profiles_resp.json()
        for profile in profiles:
            if profile.get("github_login"):
                github_to_address[profile["github_login"].lower()] = profile["address"]
        print(f"Loaded {len(github_to_address)} GitHub username -> address mappings from backend API")
    else:
        print(f"Backend API not available (status {profiles_resp.status_code}), will need manual address mapping")
except Exception as e:
    print(f"Could not fetch profiles from backend API ({e}), will need manual address mapping")

# Show unmapped assignees
all_assignees = set()
for issue in all_issues:
    for assignee in issue.get("assignees", []):
        username = assignee.get("login", "").lower()
        if username:
            all_assignees.add(username)

unmapped = [u for u in all_assignees if u not in github_to_address]
if unmapped:
    print(f"\nWarning: {len(unmapped)} assignees without address mapping:")
    for u in sorted(unmapped):
        print(f"  - {u}")
    print("These will be skipped in attestations.\n")


https://theguild-backend-e2df290d177e.herokuapp.com/
Loaded 4 GitHub username -> address mappings from backend API

  - oscarwroche
  - peteroche
  - rainwaters11
  - yash-1104github
These will be skipped in attestations.



### Generate all attestations

In [27]:
# Generate attestations from issues
# For each issue with assignees and labels, create attestations

attestations = []

# Ensure non_skill_badges is a lowercased set for quick lookup
_non_skill_badges_set = set(b.lower() for b in non_skill_badges)

for issue in all_issues:
    assignees = issue.get("assignees", [])
    labels = issue.get("labels", [])
    issue_number = issue.get("number")
    issue_title = issue.get("title", "")
    issue_url = issue.get("html_url", "")

    # Skip if no assignees or no labels
    if not assignees or not labels:
        continue

    for assignee in assignees:
        github_username = assignee.get("login", "").lower()
        if not github_username:
            continue

        recipient_address = github_to_address.get(github_username)
        if not recipient_address:
            continue  # Skip if no address mapping

        for label in labels:
            label_name = label.get("name", "")
            # Skip point labels
            if not label_name or re.match(r'^\d+pts$', label_name):
                continue
            # Skip if label is in non_skill_badges (case-insensitive)
            if label_name.lower() in _non_skill_badges_set:
                continue
            # Map label to badge, skip if mapping doesn't exist
            badge_name = git_label_to_badge.get(label_name.lower())
            if not badge_name:
                continue

            justification = f"Awarded for contributions to issue #{issue_number}: {issue_title}"
            if issue_url:
                justification += f" ({issue_url})"

            attestations.append({
                "recipient": recipient_address,
                "badgeName": badge_name,
                "justification": justification
            })

print(f"Generated {len(attestations)} attestations from {len(all_issues)} issues")
print(f"Unique recipients: {len(set(a['recipient'] for a in attestations))}")
print(f"Unique badges: {len(set(a['badgeName'] for a in attestations))}")


Generated 55 attestations from 42 issues
Unique recipients: 4
Unique badges: 15


### Save attestations to file

In [28]:
# Save attestations to date-based file and also to attestations-latest.json
from pathlib import Path

output_dir = Path("../the-guild-smart-contracts")
output_dir.mkdir(parents=True, exist_ok=True)

# Create filename with current date
date_str = datetime.now().strftime("%Y-%m-%d")
output_file = output_dir / f"attestations-{date_str}.json"

output_data = {
    "attestations": attestations
}

with open(output_file, "w") as f:
    json.dump(output_data, f, indent=2)

# Save latest attestations
latest_file = output_dir / "attestations-latest.json"
with open(latest_file, "w") as f:
    json.dump(output_data, f, indent=2)

print(f"\n✓ Saved {len(attestations)} attestations to: {output_file}")
print(f"  File size: {output_file.stat().st_size} bytes")



✓ Saved 55 attestations to: ../the-guild-smart-contracts/attestations-2025-12-03.json
  File size: 15808 bytes


Finally, run an ai prompt over the file to refactor duplicate attestations: "Here we cannot have duplicate recipient/badgeName pairs. When it is the case, just condensate the multiple justifications into one justification for one badge."

## Generate contribution tokens from issues

### Fetch closed tickets

In [3]:
import json
import re
from datetime import datetime
from collections import defaultdict

# Fetch all issues (only closed) to get complete picture
all_issues = fetch_issues(state="closed", per_page=100, max_pages=10)
print(f"Fetched {len(all_issues)} total issues (closed)")

Fetched 44 total issues (closed)


### Extract contribution labels

In [4]:
# Extract contribution labels that match the pattern "xpts"
# Extract contribution labels that match the pattern "xpts"
contributions = []

for issue in all_issues:
    issue_title = issue.get("title", "")
    for label in issue.get("labels", []):
        label_name = label.get("name", "")
        # Match labels like "10pts", "5pts", etc.
        points_match = re.match(r"^(\d+)pts$", label_name)
        if points_match:
            amount = int(points_match.group(1))
            user = None
            # Try to get GitHub handle from assignee or user
            if issue.get("assignee"):
                user = issue["assignee"].get("login", "")
            else:
                # fallback: creator of the issue
                user = issue.get("user", {}).get("login", "")
            justification = issue_title
            if user:
                contributions.append({
                    "amount": amount,
                    "githandle": user,
                    "justification": justification,
                })

print(f"\n{'='*60}")
print(f"ISSUE-BASED GITHUB CONTRIBUTION TOKENS ({len(contributions)}):")
print(f"{'='*60}")
for c in contributions:
    print(f"- {c['githandle']}: {c['amount']}pts -- \"{c['justification']}\"")
print(f"{'='*60}\n")


ISSUE-BASED GITHUB CONTRIBUTION TOKENS (13):
- tusharshah21: 40pts -- "Github handle errors"
- tusharshah21: 40pts -- "Implement SIWE for the front end"
- pheobeayo: 80pts -- "Feature: add a github handle to profile in Front end"
- pheobeayo: 40pts -- "Add doc for indexer"
- PeterOche: 80pts -- "Discord Bot foundation"
- rainwaters11: 80pts -- "Add GitHub handles to backend database (associated with profiles)"
- joelamouche: 160pts -- "Add a backend for the contribution points (api, db)"
- Teddy1792: 20pts -- "Improve landing page design with animated background"
- pheobeayo: 40pts -- "On the profile, there should be a section with the badges issued by that profile"
- tusharshah21: 20pts -- "Add copy to clipboard icons next to all ethereum addresses"
- tusharshah21: 40pts -- "Add description on the profile page"
- tusharshah21: 320pts -- "Improve SIWE logic to use dynamic, user-specific nonce - backend"
- yash-1104github: 80pts -- "Improve loading UX for lists"



### get github handles from API

In [5]:
# Optionally fetch profiles from backend API to map GitHub usernames to addresses
BACKEND_API_URL = os.getenv("BACKEND_API_URL", "http://localhost:3000")
print(BACKEND_API_URL)
github_to_address = {}

try:
    profiles_resp = requests.get(f"{BACKEND_API_URL}/profiles", timeout=5)
    if profiles_resp.status_code == 200:
        profiles = profiles_resp.json()
        for profile in profiles:
            if profile.get("github_login"):
                github_to_address[profile["github_login"].lower()] = profile["address"]
        print(f"Loaded {len(github_to_address)} GitHub username -> address mappings from backend API")
    else:
        print(f"Backend API not available (status {profiles_resp.status_code}), will need manual address mapping")
except Exception as e:
    print(f"Could not fetch profiles from backend API ({e}), will need manual address mapping")

# Show unmapped assignees
all_assignees = set()
for issue in all_issues:
    for assignee in issue.get("assignees", []):
        username = assignee.get("login", "").lower()
        if username:
            all_assignees.add(username)

unmapped = [u for u in all_assignees if u not in github_to_address]
if unmapped:
    print(f"\nWarning: {len(unmapped)} assignees without address mapping:")
    for u in sorted(unmapped):
        print(f"  - {u}")
    print("These will be skipped in attestations.\n")


https://theguild-backend-e2df290d177e.herokuapp.com/
Loaded 6 GitHub username -> address mappings from backend API

  - peteroche
  - rainwaters11
  - yash-1104github
These will be skipped in attestations.



### Generate contributions

In [6]:
import json

# Prepare mints list in expected JSON format
mints = []
for c in contributions:
    github_login = c['githandle'].lower()
    address = github_to_address.get(github_login)
    if not address:
        continue  # skip if we don't have an address
    amount = c['amount']
    justification = c['justification']
    # Prepare reason as bytes and convert to hex string ("0x...")
    # Handles possible utf-8 and encodes to hex
    reason_bytes = justification.encode("utf-8")
    reason_hex = "0x" + reason_bytes.hex()
    mints.append({
        "recipient": address,
        "amount": int(amount),
        "reason": reason_hex,
    })

out_path = "contribution-tokens-latest.json"
with open(out_path, "w") as f:
    json.dump({"mints": mints}, f, indent=2)
print(f"Wrote {len(mints)} mints to {out_path}")


Wrote 10 mints to contribution-tokens-latest.json
